In [2]:
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
import flask
from flask_cors import CORS
import os
import numpy as np

from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import entropy
import networkx as nx

In [80]:
df = pd.read_csv('./small_molecule_drugbank.csv').drop(['Unnamed: 0'],axis=1)

In [81]:
df.head()

,NAME,PAGE,IMG_URL,LOGP,PKA,MW,FORM,SOL,DESC
0,Cyclacillin,https://www.drugbank.ca/drugs/DB01000,https://www.drugbank.ca/structures/DB01000/thu...,1.31,3.30,341.4260,C15H23N3O4S,1.9000,A cyclohexylamido analog of penicillanic acid....
1,Salbutamol,https://www.drugbank.ca/drugs/DB01001,https://www.drugbank.ca/structures/DB01001/thu...,1.40,10.12,239.3107,C13H21NO3,2.1500,"Salbutamol is a short-acting, selective beta2-..."
2,Levobupivacaine,https://www.drugbank.ca/drugs/DB01002,https://www.drugbank.ca/structures/DB01002/thu...,3.60,13.62,288.4277,C18H28N2O,0.0977,Levobupivacaine is an amino-amide local anaest...
3,Cromoglicic acid,https://www.drugbank.ca/drugs/DB01003,https://www.drugbank.ca/structures/DB01003/thu...,1.92,1.77,468.3665,C23H16O11,0.0358,A chromone complex that acts by inhibiting the...
4,Ganciclovir,https://www.drugbank.ca/drugs/DB01004,https://www.drugbank.ca/structures/DB01004/thu...,-1.66,10.16,255.2306,C9H13N5O4,11.5000,An acyclovir analog that is a potent inhibitor...


In [82]:
df_yelp = pd.read_feather('./df_final_doc2topics.feather')
df_yelp.drop(['business_id'], axis=1, inplace=True)
df = df.iloc[:df_yelp.shape[0]]  # reduce num of rows to be same as df_yelp


In [83]:
data = df_yelp.drop(['name', 'is_strip', 'stars'], axis=1).as_matrix()

In [84]:
def jensen_shannon(_P, _Q):
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

# Pairwise Jensen-Shannon distance between each pair of observations based on the 18 topic-probabilities
pairwise_dist = pairwise_distances(X=data, metric=jensen_shannon)

In [85]:
df_yelp.shape

(237, 21)

In [86]:
df_yelp.columns

Index(['name', 'is_strip', 'stars', 'Topic1', 'Topic2', 'Topic3', 'Topic4',
       'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11',
       'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17',
       'Topic18'],
      dtype='object')

In [87]:
data.shape

(237, 18)

In [88]:
# find the topic most closely related to each restaurant
topic_closest_ind = np.argmax(data, axis=1)

topic_names_ord = ['Cost & Quality', 'Bars', 'Casino Hotel', 'Fine Dining', 'Asian', 'Pizza', 'Steakhouse', 
                   'Italian', 'Coffee Shop', 'High Customer Satisfaction', 'Night Club', 'Wait Time', 'Mexican', 
                   'Lunch', 'Sushi', 'Fast Food', 'Breakfast', 'Low Customer Satisfaction']

# names of topics most closely related to each restaurant (ordered by the order of restaurants in df)
topic_closest = [topic_names_ord[ind] for ind in topic_closest_ind]

In [89]:
# find the second most closely related topic for each restaurant
topic_closest_ind = np.argsort(data, axis=1)[:,::-1][:,:3]

In [90]:
topic_closest_ind

array([[ 9, 12, 13],
       [ 4, 15,  1],
       [15,  1,  3],
       [16, 17,  8],
       [ 1, 16, 15],
       [ 3,  7, 16],
       [10,  1,  8],
       [16,  1,  6],
       [16, 15, 11],
       [ 3,  1, 16],
       [13, 16, 12],
       [11,  1,  7],
       [13, 16,  6],
       [ 5,  1, 15],
       [ 1, 15, 10],
       [16,  8,  1],
       [ 3, 11, 16],
       [ 2, 15, 10],
       [ 3,  1, 16],
       [ 4, 15,  0],
       [13,  4, 15],
       [ 5,  1, 15],
       [16, 15,  8],
       [ 9, 11, 14],
       [16, 15,  2],
       [ 6, 15, 11],
       [ 9, 11, 14],
       [14,  3, 16],
       [16,  1, 17],
       [ 3,  9, 14],
       [ 0,  9, 16],
       [ 9,  3, 14],
       [ 4,  0, 15],
       [10, 15, 11],
       [ 9, 17, 13],
       [ 9,  4, 14],
       [16,  8,  1],
       [14,  6, 15],
       [16,  8, 17],
       [12, 15,  2],
       [ 1, 16,  8],
       [16,  8,  1],
       [16, 13,  5],
       [ 4, 11, 14],
       [ 3,  7,  5],
       [ 2, 15, 10],
       [12, 16,  5],
       [ 6, 1

In [91]:
data[:5, :]

array([[3.18641067e-02, 1.08591351e-03, 5.28910523e-03, 7.90708326e-03,
        6.25351677e-03, 1.52236829e-02, 1.35446731e-02, 3.00209783e-03,
        7.63574149e-04, 6.19946301e-01, 1.43517327e-05, 1.43517327e-05,
        1.02889627e-01, 9.05911177e-02, 1.16093159e-02, 5.41634597e-02,
        1.36089586e-02, 2.22287718e-02],
       [8.26981734e-04, 8.12570900e-02, 2.91083590e-03, 5.30772582e-02,
        5.75447917e-01, 5.39200800e-03, 5.04531257e-04, 7.93924090e-03,
        9.34654661e-03, 3.64685408e-03, 3.42760310e-02, 1.35348085e-02,
        1.62074268e-02, 6.14185352e-03, 6.79230876e-03, 1.73229292e-01,
        3.96069605e-03, 5.50837489e-03],
       [7.08767306e-03, 2.22557515e-01, 9.24696587e-03, 1.07580289e-01,
        3.46804434e-03, 9.97224543e-03, 1.67127475e-02, 1.66569445e-02,
        4.63641062e-02, 8.36915337e-03, 4.05711085e-02, 1.02029666e-01,
        2.82413922e-02, 1.22123342e-02, 9.58541036e-03, 3.18068594e-01,
        2.18892936e-02, 1.93865281e-02],
       [4.860

In [92]:
topic_closest_ind[:5, :]

array([[ 9, 12, 13],
       [ 4, 15,  1],
       [15,  1,  3],
       [16, 17,  8],
       [ 1, 16, 15]])

In [93]:
topic_closest_ind

array([[ 9, 12, 13],
       [ 4, 15,  1],
       [15,  1,  3],
       [16, 17,  8],
       [ 1, 16, 15],
       [ 3,  7, 16],
       [10,  1,  8],
       [16,  1,  6],
       [16, 15, 11],
       [ 3,  1, 16],
       [13, 16, 12],
       [11,  1,  7],
       [13, 16,  6],
       [ 5,  1, 15],
       [ 1, 15, 10],
       [16,  8,  1],
       [ 3, 11, 16],
       [ 2, 15, 10],
       [ 3,  1, 16],
       [ 4, 15,  0],
       [13,  4, 15],
       [ 5,  1, 15],
       [16, 15,  8],
       [ 9, 11, 14],
       [16, 15,  2],
       [ 6, 15, 11],
       [ 9, 11, 14],
       [14,  3, 16],
       [16,  1, 17],
       [ 3,  9, 14],
       [ 0,  9, 16],
       [ 9,  3, 14],
       [ 4,  0, 15],
       [10, 15, 11],
       [ 9, 17, 13],
       [ 9,  4, 14],
       [16,  8,  1],
       [14,  6, 15],
       [16,  8, 17],
       [12, 15,  2],
       [ 1, 16,  8],
       [16,  8,  1],
       [16, 13,  5],
       [ 4, 11, 14],
       [ 3,  7,  5],
       [ 2, 15, 10],
       [12, 16,  5],
       [ 6, 1

In [94]:
topic_names_ord = ['Cost & Quality', 'Bars', 'Casino Hotel', 'Fine Dining', 'Asian', 'Pizza', 'Steakhouse', 
                   'Italian', 'Coffee Shop', 'High Customer Satisfaction', 'Night Club', 'Wait Time', 'Mexican', 
                   'Lunch', 'Sushi Restaurant', 'Fast Food', 'Breakfast', 'Low Customer Satisfaction']

# names of topics most closely related to each restaurant (ordered by the order of restaurants in df)
topic_closest = [[topic_names_ord[ind] for ind in rest] for rest in topic_closest_ind]
# topic_closest = [topic_names_ord[ind] for ind in topic_closest_ind]

In [95]:
topic_closest

[['High Customer Satisfaction', 'Mexican', 'Lunch'],
 ['Asian', 'Fast Food', 'Bars'],
 ['Fast Food', 'Bars', 'Fine Dining'],
 ['Breakfast', 'Low Customer Satisfaction', 'Coffee Shop'],
 ['Bars', 'Breakfast', 'Fast Food'],
 ['Fine Dining', 'Italian', 'Breakfast'],
 ['Night Club', 'Bars', 'Coffee Shop'],
 ['Breakfast', 'Bars', 'Steakhouse'],
 ['Breakfast', 'Fast Food', 'Wait Time'],
 ['Fine Dining', 'Bars', 'Breakfast'],
 ['Lunch', 'Breakfast', 'Mexican'],
 ['Wait Time', 'Bars', 'Italian'],
 ['Lunch', 'Breakfast', 'Steakhouse'],
 ['Pizza', 'Bars', 'Fast Food'],
 ['Bars', 'Fast Food', 'Night Club'],
 ['Breakfast', 'Coffee Shop', 'Bars'],
 ['Fine Dining', 'Wait Time', 'Breakfast'],
 ['Casino Hotel', 'Fast Food', 'Night Club'],
 ['Fine Dining', 'Bars', 'Breakfast'],
 ['Asian', 'Fast Food', 'Cost & Quality'],
 ['Lunch', 'Asian', 'Fast Food'],
 ['Pizza', 'Bars', 'Fast Food'],
 ['Breakfast', 'Fast Food', 'Coffee Shop'],
 ['High Customer Satisfaction', 'Wait Time', 'Sushi Restaurant'],
 ['Break

In [96]:
vis = pd.read_pickle('vis.pkl')

In [97]:
def get_relevant_words(vis,lam=0.3,topn=10):
    a = vis.topic_info
    a['finalscore'] = a['logprob']*lam+(1-lam)*a['loglift']
    a = a.loc[:,['Category','Term','finalscore']].groupby(['Category'])\
    .apply(lambda x: x.sort_values(by='finalscore',ascending=False).head(topn))
    a = a.loc[:,'Term'].reset_index().loc[:,['Category','Term']]
    a = a[a['Category']!='Default']
    a = a.to_dict('split')['data']
    d ={}
    for k,v in a: 
        if k not in d.keys():
            d[k] =set()
            d[k].add(v)
        else:
            d[k].add(v)
    finalData = pd.DataFrame([],columns=['Topic','words with Relevance'])
    finalData['Topic']=d.keys()
    finalData['words with Relevance']=d.values()
    return finalData

In [98]:
def get_top_n_words_list(num_topics, vis, lam=0.6, topn=5):
    """returns a sorted list of top n words, where the list follows the order Topic 1, ..., Topic n.
    Each element of the list is a string composed of a list of the top n words
    num_topics: number of topics
    vis: pyLDAvis object
    lam: relevance value
    topn: number of topics
    """
    topic_ids_ordered = ['Topic' + str(num) for num in range(1, num_topics + 1)]
    top_topic_words_df = get_relevant_words(vis, lam, topn)
    top_topic_words_df.set_index('Topic', drop=True, inplace=True)
    top_topic_words = [top_topic_words_df.loc[topic_id]['words with Relevance'] for topic_id in topic_ids_ordered]
    top_topic_words_display = [', '.join(words) for words in top_topic_words]
    return top_topic_words_display

In [99]:
# sample usage and output
len(get_top_n_words_list(num_topics=18, vis=vis, lam=0.6, topn=3))

18

In [100]:
top3words = get_top_n_words_list(num_topics=18, vis=vis, lam=0.6, topn=3)

In [101]:
top3words

['good, food, price',
 'bar, beer, drink',
 'buffet, stay, hotel',
 'dessert, restaurant, dinner',
 'rice, thai, noodle',
 'slice, wing, pizza',
 'cook, steakhouse, steak',
 'bread, pasta, salad',
 'chocolate, tea, coffee',
 'food, service, great',
 'business, door, walk',
 'wait, table, minute',
 'taco, mexican, burrito',
 'location, place, lunch',
 'roll, fish, sushi',
 'burger, fry, shake',
 'sandwich, egg, chicken',
 'bad, order, tell']

In [102]:
topic_names_ord

['Cost & Quality',
 'Bars',
 'Casino Hotel',
 'Fine Dining',
 'Asian',
 'Pizza',
 'Steakhouse',
 'Italian',
 'Coffee Shop',
 'High Customer Satisfaction',
 'Night Club',
 'Wait Time',
 'Mexican',
 'Lunch',
 'Sushi Restaurant',
 'Fast Food',
 'Breakfast',
 'Low Customer Satisfaction']

In [103]:
topic_relwords_mapper = {}
for i in range(len(topic_names_ord)):
    topic_relwords_mapper[topic_names_ord[i]] = top3words[i]


In [104]:
topic_closest[0]

['High Customer Satisfaction', 'Mexican', 'Lunch']

In [105]:
topic1_for_disp = []
topic2_for_disp = []
topic3_for_disp = []
for review in topic_closest:
    str1 = review[0] + ': ' + topic_relwords_mapper[review[0]]
    topic1_for_disp.append(str1)
    str2 = review[1] + ': ' + topic_relwords_mapper[review[1]]
    topic2_for_disp.append(str2)
    str3 = review[2] + ': ' + topic_relwords_mapper[review[2]]
    topic3_for_disp.append(str3)

In [106]:
topic2_for_disp

['Mexican: taco, mexican, burrito',
 'Fast Food: burger, fry, shake',
 'Bars: bar, beer, drink',
 'Low Customer Satisfaction: bad, order, tell',
 'Breakfast: sandwich, egg, chicken',
 'Italian: bread, pasta, salad',
 'Bars: bar, beer, drink',
 'Bars: bar, beer, drink',
 'Fast Food: burger, fry, shake',
 'Bars: bar, beer, drink',
 'Breakfast: sandwich, egg, chicken',
 'Bars: bar, beer, drink',
 'Breakfast: sandwich, egg, chicken',
 'Bars: bar, beer, drink',
 'Fast Food: burger, fry, shake',
 'Coffee Shop: chocolate, tea, coffee',
 'Wait Time: wait, table, minute',
 'Fast Food: burger, fry, shake',
 'Bars: bar, beer, drink',
 'Fast Food: burger, fry, shake',
 'Asian: rice, thai, noodle',
 'Bars: bar, beer, drink',
 'Fast Food: burger, fry, shake',
 'Wait Time: wait, table, minute',
 'Fast Food: burger, fry, shake',
 'Fast Food: burger, fry, shake',
 'Wait Time: wait, table, minute',
 'Fine Dining: dessert, restaurant, dinner',
 'Bars: bar, beer, drink',
 'High Customer Satisfaction: food

In [107]:
len(topic1_for_disp)

237

In [108]:
topic_relwords_mapper['Italian']

'bread, pasta, salad'

In [109]:
threshold2k ={
    0.55: 0.7,
    0.56: 0.9,
    0.57: 0.3,
    0.58: 5,
    0.59: 2,
    0.6: 5,
    0.61: 5,
    0.62: 5
}

In [110]:
# arbitrary threshold for deciding whether 2 observations are 'similar' or not
threshold_all = [0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62]
def th_mark(x):
    if x==np.min(threshold_all):
        return 'Low'
    elif x==np.max(threshold_all):
        return 'High'
    else:
        return ''
    
threshold_mark = {str(th):th_mark(th) for th in threshold_all}
adjacency = [np.where(pairwise_dist > threshold, 1, 0) for threshold in threshold_all]

In [111]:
# map threshold value to adjacency matrix
thresh_to_adj = {thresh: adj for thresh, adj in zip(threshold_all, adjacency)}

In [112]:
def create_graph(adj):
    # input: adjaccency matrix
    # returns a graph with the isolates removed
    G = nx.from_numpy_matrix(adj)
    isolates = list(nx.isolates(G))
    G.remove_nodes_from(isolates)
    return G

In [113]:
# map threshold value to graph
thresh_to_graph = {thresh: create_graph(adj) for thresh, adj in zip(threshold_all, adjacency)}

In [114]:
# extract node positions
fruchterman_iter = 1000

# map threshold values to positions of nodes
thresh_to_pos = {}

for thresh in thresh_to_graph:
    graph = nx.fruchterman_reingold_layout(thresh_to_graph[thresh], k = threshold2k[thresh], iterations=fruchterman_iter)
    thresh_to_pos[thresh] = graph

In [115]:
thresh_to_XnYn = {}
for thresh in thresh_to_pos:
    pos = thresh_to_pos[thresh]
    # define lists of node coordinates
    Xn_strip = [pos[k][0] for k in sorted(pos.keys()) if k in df_yelp.index[df_yelp.is_strip == True]]
    Yn_strip = [pos[k][1] for k in sorted(pos.keys()) if k in df_yelp.index[df_yelp.is_strip == True]]
    Xn_notstrip = [pos[k][0] for k in sorted(pos.keys()) if k in df_yelp.index[df_yelp.is_strip == False]]
    Yn_notstrip = [pos[k][1] for k in sorted(pos.keys()) if k in df_yelp.index[df_yelp.is_strip == False]]
    thresh_to_XnYn[thresh] = (Xn_strip, Yn_strip, Xn_notstrip, Yn_notstrip)

In [116]:
# concatenating the 2 dataframes
df = pd.concat([df, df_yelp], axis=1)

In [117]:
df.stars = df.stars.astype(str)
df.stars = df.stars + ' stars'

In [118]:
df['temp_id'] = df.index
df['topic1_for_disp'] = topic1_for_disp
df['topic2_for_disp'] = topic2_for_disp
df['topic3_for_disp'] = topic3_for_disp

# map threshold value to a list of nodes left over in the graph after isolate removal
thresh_to_nodenums = {}
for thresh in thresh_to_graph:
    graph = thresh_to_graph[thresh]
    nodenums = list(graph.nodes())
    thresh_to_nodenums[thresh] = nodenums
    
# format of thresh_to_XnYn's output:
# thresh_to_XnYn[thresh] = (Xn_strip, Yn_strip, Xn_notstrip, Yn_notstrip)

stacked_df = []
for thresh in thresh_to_XnYn:
    # for each threshold, create a copy of df
    df_temp = df.copy()
    
    # positions of nodes for the graph
    Xn_strip, Yn_strip, Xn_notstrip, Yn_notstrip = thresh_to_XnYn[thresh]
    # nodes left after removing isolates
    nodenums = thresh_to_nodenums[thresh]
    
    df_temp['threshold'] = thresh
    # initialize Xn and Yn
    df_temp['Xn'] = np.nan
    df_temp['Yn'] = np.nan
    # fill in X and Y positions of nodes for non-isolate nodes
    df_temp.loc[(df_temp.is_strip == True) & (df_temp.temp_id.isin(nodenums)), 'Xn'] = Xn_strip
    df_temp.loc[(df_temp.is_strip == True) & (df_temp.temp_id.isin(nodenums)), 'Yn'] = Yn_strip
    df_temp.loc[(df_temp.is_strip == False) & (df_temp.temp_id.isin(nodenums)), 'Xn'] = Xn_notstrip
    df_temp.loc[(df_temp.is_strip == False) & (df_temp.temp_id.isin(nodenums)), 'Yn'] = Yn_notstrip
    
    stacked_df.append(df_temp)
    
# vertically stack the dataframes in stacked_df
df_final = pd.concat(stacked_df, axis=0, ignore_index=True)

In [119]:
df_final.shape

(1896, 37)

In [120]:
df_final.dtypes

NAME                object
PAGE                object
IMG_URL             object
LOGP               float64
PKA                float64
MW                 float64
FORM                object
SOL                float64
DESC                object
name                object
is_strip            object
stars               object
Topic1             float64
Topic2             float64
Topic3             float64
Topic4             float64
Topic5             float64
Topic6             float64
Topic7             float64
Topic8             float64
Topic9             float64
Topic10            float64
Topic11            float64
Topic12            float64
Topic13            float64
Topic14            float64
Topic15            float64
Topic16            float64
Topic17            float64
Topic18            float64
temp_id              int64
topic1_for_disp     object
topic2_for_disp     object
topic3_for_disp     object
threshold          float64
Xn                 float64
Yn                 float64
d

In [121]:
df_final.drop(['NAME', 'FORM', 'SOL', 'DESC', 'LOGP', 'PKA', 'MW'], axis=1, inplace=True)

In [122]:
df_final.rename(columns={'name': 'NAME'}, inplace=True)

In [123]:
df_final.dtypes

PAGE                object
IMG_URL             object
NAME                object
is_strip            object
stars               object
Topic1             float64
Topic2             float64
Topic3             float64
Topic4             float64
Topic5             float64
Topic6             float64
Topic7             float64
Topic8             float64
Topic9             float64
Topic10            float64
Topic11            float64
Topic12            float64
Topic13            float64
Topic14            float64
Topic15            float64
Topic16            float64
Topic17            float64
Topic18            float64
temp_id              int64
topic1_for_disp     object
topic2_for_disp     object
topic3_for_disp     object
threshold          float64
Xn                 float64
Yn                 float64
dtype: object

In [126]:
df_final.head(10).T

,0,1,2,3,4,5,6,7,8,9
PAGE,https://www.drugbank.ca/drugs/DB01000,https://www.drugbank.ca/drugs/DB01001,https://www.drugbank.ca/drugs/DB01002,https://www.drugbank.ca/drugs/DB01003,https://www.drugbank.ca/drugs/DB01004,https://www.drugbank.ca/drugs/DB01005,https://www.drugbank.ca/drugs/DB01009,https://www.drugbank.ca/drugs/DB01010,https://www.drugbank.ca/drugs/DB01013,https://www.drugbank.ca/drugs/DB01014
IMG_URL,https://www.drugbank.ca/structures/DB01000/thu...,https://www.drugbank.ca/structures/DB01001/thu...,https://www.drugbank.ca/structures/DB01002/thu...,https://www.drugbank.ca/structures/DB01003/thu...,https://www.drugbank.ca/structures/DB01004/thu...,https://www.drugbank.ca/structures/DB01005/thu...,https://www.drugbank.ca/structures/DB01009/thu...,https://www.drugbank.ca/structures/DB01010/thu...,https://www.drugbank.ca/structures/DB01013/thu...,https://www.drugbank.ca/structures/DB01014/thu...
NAME,Jaburritos,Le Thai,Sugar Factory,Rollin' Smoke BBQ,PublicUs,Scarpetta,Lazy Dog Restaurant & Bar,Bruxie,Cracker Barrel Old Country Store,Carson Kitchen
is_strip,True,False,True,False,False,True,False,True,False,False
stars,4.5 stars,4.0 stars,3.5 stars,4.0 stars,4.5 stars,4.0 stars,4.0 stars,4.5 stars,3.0 stars,4.5 stars
Topic1,0.0318641,0.000826982,0.00708767,0.00486025,0.00891615,0.00313926,0.00970048,0.00556002,0.00176765,0.0191866
Topic2,0.00108591,0.0812571,0.222558,0.00043856,0.428269,0.00543957,0.143152,0.0411154,0.00374424,0.165109
Topic3,0.00528911,0.00291084,0.00924697,0.021188,0.0109089,9.52599e-06,0.00919097,0.0134769,0.0765606,5.24852e-06
Topic4,0.00790708,0.0530773,0.10758,0.011603,0.0168612,0.750131,0.0370581,0.000569014,0.00428274,0.277254
Topic5,0.00625352,0.575448,0.00346804,0.0425554,0.00183876,0.00648876,0.00841399,0.000650337,9.64004e-06,0.0172299


In [127]:
df_final.to_feather('df_final.feather')

In [128]:
import pickle
with open('business_names.pkl', 'wb') as f:
    pickle.dump(df_final.NAME.tolist(), f)
    

In [129]:
with open('thresh_to_pos.pkl', 'wb') as f:
    pickle.dump(thresh_to_pos, f)

with open('thresh_to_XnYn.pkl', 'wb') as f:
    pickle.dump(thresh_to_XnYn, f)
    
with open('thresh_to_graph.pkl', 'wb') as f:
    pickle.dump(thresh_to_graph, f)

In [3]:
df = pd.read_csv('../data/topic2word.csv')

In [4]:
df.to_pickle('../data/topic2word.pkl')

In [6]:
df.columns.tolist()

['Unnamed: 0',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',

In [7]:
df.columns = df.columns.tolist()

In [9]:
a1 = [1,2,3,4]
[e*10 for e in a1]

[10, 20, 30, 40]